**Scenario 2: Educational Tools**

CodeGenie can be integrated into educational platforms to assist students in learning programming languages. Students can input their code assignments or projects, and CodeGenie will provide insights into code efficiency, syntax correctness, and logical flow. Additionally, it can offer suggestions for improvement and highlight specific parts of the code that are particularly well-written or problematic. This helps students learn to write clean, efficient, and error-free code, improving their coding skills and technic



In [ ]:
!pip install git+https://github.com/huggingface/transformers.git@refs/pull/25740/head accelerate
!pip install streamlit -q
!pip install accelerate
!pip install bitsandbytes


  Cloning https://github.com/huggingface/transformers.git (to revision refs/pull/25740/head) to /tmp/pip-req-build-dgjimyhy
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-dgjimyhy


In [ ]:
%%writefile CodeModel.py

# CodeModel.py

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import torch
import streamlit as st
import re

@st.cache_resource
def load_model():
    model_id = "codellama/CodeLlama-7b-Instruct-hf"
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    bnb_config = BitsAndBytesConfig(load_in_8bit=True)

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        quantization_config=bnb_config,
        torch_dtype=torch.float16
    )

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        device_map="auto"
    )
    return pipe, tokenizer

pipe, tokenizer = load_model()



def generate_code(user):

    def truncate_to_last_sentence(text):
        # Remove hanging bullet like "19." at end
        match = re.search(r"(.*?)(\n\d+\.\s*)?$", text.strip(), re.DOTALL)
        if match:
            text = match.group(1)

        # Now trim to last full sentence
        if '.' in text:
            last_period = text.rfind('.')
            return text[:last_period + 1].strip()
        return text.strip()


    # system = "You are CodeGenie, a concise programming tutor. Summarize what the code does, evaluate logic and style, then suggest clear fixes."
    system = "You are CodeGenie, a concise programming tutor. Summarize what the code does, evaluate logic and style, then suggest clear fixes."
    '''user = """Analyze this Python function and give feedback:

    def add(a, b):
      return a + b"""'''
    prompt = f"<s><<SYS>>\n{system}\n<</SYS>>\n\n{user}"

    prompt_tokens = tokenizer(prompt, return_tensors="pt").input_ids.shape[-1]
    max_total_tokens = 256
    max_new_tokens = max(32, max_total_tokens - prompt_tokens)

    sequences = pipe(
        prompt,
        do_sample=True,
        top_k=10,
        temperature=0.1,
        top_p=0.95,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=max_new_tokens,
        add_special_tokens=False
    )

    generated = sequences[0]['generated_text']
    if prompt in generated:
        result = generated.replace(prompt, "").strip()
    else:
        # In case model doesn't echo prompt, just return from the first new line
        result = generated.split('\n', 1)[-1].strip()

    result = truncate_to_last_sentence(result)

    return result

In [ ]:
%%writefile app.py

import streamlit as st
import time
from CodeModel import generate_code
st.header("CodeGenie: AI-powered code assistant")

with st.form("my_form"):
    user_input = st.text_area("Enter your text prompt below and click the button to submit.")
    submit = st.form_submit_button(label="Submit text prompt")

if submit:
    with st.spinner(text="Generating code... It may take some time"):
        code = generate_code(user=user_input)
        print(code)
        st.code(code, language='python')
    st.sidebar.markdown("## Guide")
    st.sidebar.info("This tool uses CodeLlama 7B parameters")

In [ ]:
!wget -q -O - ipv4.icanhazip.com

**Copy that number and paste it on the website as password on next cell**

**Click "your url is:" link and paste that number**


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501